In [1]:
# Restart kernel and reload modules
import importlib
import sys
if 'fugu.bricks.lca_bricks' in sys.modules:
    importlib.reload(sys.modules['fugu.bricks.lca_bricks'])
if 'fugu.bricks' in sys.modules:
    importlib.reload(sys.modules['fugu.bricks'])


In [2]:
# Force reload the module
import sys
import importlib

# Remove modules from cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('fugu')]
for module in modules_to_remove:
    del sys.modules[module]

# Re-import everything fresh
from fugu.bricks import LCABrick
from fugu.backends import slca_Backend  
import numpy as np
from fugu import Scaffold

Phi_demo = np.array([
    [0.3313, 0.8148, 0.4364],
    [0.8835, 0.3621, 0.2182],
    [0.3313, 0.4527, 0.8729],
], dtype=float)
s_demo = np.array([0.5, 1.0, 1.5], dtype=float)
lam_demo = 0.1

scaffold = Scaffold()

scaffold.add_brick(
    LCABrick(Phi=Phi_demo),
    output=True
)

scaffold.lay_bricks()

In [3]:
graph = scaffold.graph
print(f"Graph has {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges")

lca_neurons = [n for n in graph.nodes() if 'neuron_' in n]
print(f"Found {len(lca_neurons)} LCA neurons:")
for i, neuron in enumerate(lca_neurons): 
    print(f"  {neuron}: {graph.nodes[neuron]}")

print(f"\nExample edges (inhibitory connections):")
for i, (src, dst, data) in enumerate(graph.edges(data=True)):
    print(f"  {src} -> {dst}: weight={data.get('weight', 'N/A')}, delay={data.get('delay', 'N/A')}")

Graph has 4 nodes and 6 edges
Found 3 LCA neurons:
  Brick-0:neuron_0: {'index': 0, 'threshold': 1.0, 'potential': 0.0, 'p': 1.0, 'brick': 'Brick-0', 'neuron_number': 1}
  Brick-0:neuron_1: {'index': 1, 'threshold': 1.0, 'potential': 0.0, 'p': 1.0, 'brick': 'Brick-0', 'neuron_number': 2}
  Brick-0:neuron_2: {'index': 2, 'threshold': 1.0, 'potential': 0.0, 'p': 1.0, 'brick': 'Brick-0', 'neuron_number': 3}

Example edges (inhibitory connections):
  Brick-0:neuron_0 -> Brick-0:neuron_1: weight=0, delay=1
  Brick-0:neuron_0 -> Brick-0:neuron_2: weight=0, delay=1
  Brick-0:neuron_1 -> Brick-0:neuron_0: weight=0, delay=1
  Brick-0:neuron_1 -> Brick-0:neuron_2: weight=0, delay=1
  Brick-0:neuron_2 -> Brick-0:neuron_0: weight=0, delay=1
  Brick-0:neuron_2 -> Brick-0:neuron_1: weight=0, delay=1


In [6]:
backend = slca_Backend()
backend.compile(
    scaffold=scaffold,
    compile_args={
        'y': s_demo,
        'Phi': Phi_demo,
        'lam': lam_demo,
        'dt': 1e-3,
        'tau_syn': 1,
        'T_steps': 100000,
        't0_steps': 1000,
    }
)

result = backend.run(
)
"""

self.Phi = compile_args.get('Phi', None)
self.y_obs = compile_args.get('y', None)
self.K = compile_args.get('K', None)
self.lam = float(compile_args.get('lam', 0.1))
self.dt = float(compile_args.get('dt', 1e-4))
self.tau= float(compile_args.get('tau_syn', 1e-2))
self.T_steps = int(compile_args.get('T_steps', 1000))
self.t0_steps = int(compile_args.get('t0_steps', max (1, self.T_steps // 10)))
self.unit_area = bool(compile_args.get('unit_area', True))

"""

"\n\nself.Phi = compile_args.get('Phi', None)\nself.y_obs = compile_args.get('y', None)\nself.K = compile_args.get('K', None)\nself.lam = float(compile_args.get('lam', 0.1))\nself.dt = float(compile_args.get('dt', 1e-4))\nself.tau= float(compile_args.get('tau_syn', 1e-2))\nself.T_steps = int(compile_args.get('T_steps', 1000))\nself.t0_steps = int(compile_args.get('t0_steps', max (1, self.T_steps // 10)))\nself.unit_area = bool(compile_args.get('unit_area', True))\n\n"

In [7]:
result

{'a_tail': array([0.68402167, 0.        , 1.21029842]),
 'a_rate': array([0.  , 0.68, 0.01, 1.21]),
 'counts': array([  0,  68,   1, 121]),
 'x_hat': array([0.75477743, 0.86839117, 1.28306989]),
 'b': array([1.54602917, 1.44858423, 1.74574074]),
 'W': array([[0.        , 0.73982169, 0.62651876],
        [0.73982169, 0.        , 0.82976598],
        [0.62651876, 0.82976598, 0.        ]])}